In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import asyncio

In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
p = open("../data/RuyLopezMarshall.pgn")

In [4]:
games_id = []
games = []

while True:
    offset = p.tell()
    headers = chess.pgn.read_headers(p)
    if headers is None:
        break
    games_id.append(offset)

for offset in games_id:
    p.seek(offset)
    games.append(chess.pgn.read_game(p))

In [5]:
dados = []
count = -1

for game in games[:2]:
    count += 1
    row = {}
    head = game.headers

    if head['Site'].endswith(' INT') or head['Site'] == '?':
        continue

    row['id'] = games_id[count]
    row['Event'] = head['Event']
    row['Site'] = head['Site']
    row['Date'] = head['Date'].split('.')[0]
    row['White'] = head['White']
    row['Black'] = head['Black']
    row['Result'] = head['Result']
    row['BlackElo'] = head['BlackElo']
    row['ECO'] = 'C89'
    row['WhiteElo'] = head['WhiteElo']
    row['White Short Castle'] = 0
    row['White Long Castle'] = 0
    row['Black Short Castle'] = 0
    row['Black Long Castle'] = 0
    row['Opposite side castling'] = 0
    row['Evals'] = []

    # quem ganhou
    if head['Result'] == '0-1':
        row['WhiteWon'] = 0
        row['BlackWon'] = 1
    elif head['Result'] == '1-0':
        row['WhiteWon'] = 1
        row['BlackWon'] = 0
    else:
        row['WhiteWon'] = 0
        row['BlackWon'] = 0

    # qntd de moves
    final_position = game[0].end().board()
    row['Moves'] = final_position.fullmove_number

    # qntd de peças restantes
    final_fen = final_position.board_fen()
    white_count = 0
    black_count = 0
    white_pieces = ['P', 'N', 'B', 'R', 'Q']
    black_pieces = ['p', 'n', 'b', 'r', 'q']
    white_has_queen = 0
    black_has_queen = 0

    for char in range(0, len(final_fen)):
        if final_fen[char] in white_pieces:
            white_count += 1
        elif final_fen[char] in black_pieces:
            black_count += 1

        if final_fen[char] == 'Q':
            white_has_queen = 1
        elif final_fen[char] == 'q':
            black_has_queen = 1

    row['WhiteFinalPiecesCount'] = white_count
    row['BlackFinalPiecesCount'] = black_count
    row['FinalPiecesCount'] = white_count + black_count

    if white_has_queen == 0 and black_has_queen == 0:
        row['QueenTrade'] = 1
    else:
        row['QueenTrade'] = 0

    board = game[0].board()
    info = eng.analyse(board, chess.engine.Limit(time=0.1))
    row['Evals'].append(tuple([board.peek().uci(), info['score'].white().score(mate_score=100000), info['pv'][0].uci()]))

    flag_wking_move = 0
    flag_bking_move = 0

    for move in game[0].mainline_moves():
        m = move.uci()

        board.push(move)
        info = eng.analyse(board, chess.engine.Limit(time=0.1))
        row['Evals'].append((tuple([m, info['score'].white().score(mate_score=100000), info['pv'][0].uci()])))

        if m[:2] == 'e1' and flag_wking_move == 0:
            flag_wking_move = 1

            if m == 'e1g1':
                row['White Short Castle'] = 1
            elif m == 'e1c1':
                row['White Long Castle'] = 1

        elif m[:2] == 'e8' and flag_bking_move == 0:
            flag_bking_move = 1

            if m == 'e8g8':
                row['Black Short Castle'] = 1
            elif m == 'e8c8':
                row['Black Long Castle'] = 1

        if (row['White Short Castle'] == 1 and row['Black Long Castle'] == 1) or \
                (row['Black Short Castle'] == 1 and row['White Long Castle'] == 1):
            row['Opposite side castling'] = 1
    
    dados.append(row)

eng.quit()
df = pd.DataFrame(dados)


In [6]:
print(df['Evals'][0])

[('e2e4', 35, 'c7c5'), ('e7e5', 47, 'g1f3'), ('g1f3', 35, 'b8c6'), ('b8c6', 43, 'f1b5'), ('f1b5', 36, 'g8f6'), ('a7a6', 59, 'b5a4'), ('b5a4', 52, 'g8f6'), ('g8f6', 51, 'e1g1'), ('e1g1', 52, 'b7b5'), ('f8e7', 71, 'f1e1'), ('f1e1', 43, 'b7b5'), ('b7b5', 46, 'a4b3'), ('a4b3', 35, 'e8g8'), ('e8g8', 51, 'c2c3'), ('c2c3', 27, 'd7d5'), ('d7d5', 37, 'd2d4'), ('e4d5', 28, 'f6d5'), ('f6d5', 14, 'a2a4'), ('d2d4', -9, 'e5d4'), ('e5d4', -1, 'f3d4'), ('c3d4', -25, 'c8g4'), ('e7b4', 71, 'c1g5'), ('c1d2', 21, 'b4d2'), ('c8g4', 70, 'h2h3'), ('b1c3', -36, 'd5f6'), ('d5f6', -33, 'd2e3'), ('d2e3', -16, 'f8e8'), ('g4f3', -37, 'd1f3'), ('g2f3', -114, 'f8e8'), ('d8d7', -102, 'a2a3'), ('d4d5', -181, 'c6e7'), ('c6e7', -124, 'e3g5'), ('e3g5', -183, 'd7h3'), ('d7h3', -296, 'd1d4'), ('g5f6', -356, 'b4d6'), ('g7f6', -30, 'd1d4'), ('d1d4', -39, 'a6a5'), ('b4d6', -33, 'd4g4'), ('d4g4', -33, 'h3g4'), ('h3g4', -40, 'f3g4'), ('f3g4', -39, 'f6f5'), ('e7g6', 0, 'c3e4'), ('c3e4', -3, 'd6e5'), ('d6e5', 1, 'a1c1'), ('a1b1',

In [7]:
# corrigir places/Sites
def string_analyzer(row):
    s = row['Site']
    splitted_s = s.split(' ')
    first_part = splitted_s[:-1]
    last_part = splitted_s[-1]
    city = ''

    if last_part.isupper() and len(splitted_s) > 1:
        for w in first_part:
            city += w + ' '
    else:
        city = s

    row['Site'] = city

    return row


df = df.apply(lambda row: string_analyzer(row=row), axis=1)
